In [2]:
from os import chdir, path, getcwd
for i in range(10):
    if path.isfile("checkcwd"):
        break
    chdir(path.pardir)
if path.isfile("checkcwd"):
    pass
else:
    raise Exception("Something went wrong. cwd=" + getcwd())

In [3]:
import pandas as pd
data = pd.read_pickle('database/cleaned_data.pkl')
data.dropna()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,full_datetime


In [4]:
kill_data = data.groupby(['country', 'country_txt', 'iyear'])['nkill'].sum().reset_index()
kill_data = kill_data.rename(columns={'country_txt': 'Country'})

log_bin_edges = [0, 3, 10, 30, 100, 300, 1000, 3000, 10000, 30000]
log_bin_labels = ['0-3', '3-10', '10-30', '30-100', '100-300', '300-1000', '1000-3000', '3000-10000', '10000-30000']
kill_data['nkill_log'] = pd.cut(kill_data['nkill'], bins=log_bin_edges, labels=log_bin_labels, include_lowest=True)


kill_data = kill_data.sort_values(['iyear', 'nkill'])
kill_data

,country,Country,iyear,nkill,nkill_log
160,AUS,Australia,1970,0.0,0-3
272,BEL,Belgium,1970,0.0,0-3
449,BOL,Bolivia,1970,0.0,0-3
777,COL,Colombia,1970,0.0,0-3
910,DEU,East Germany (GDR),1970,0.0,0-3
...,...,...,...,...,...
3476,SYR,Syria,2020,973.0,300-1000
1256,ETH,Ethiopia,2020,1330.0,1000-3000
763,COD,Democratic Republic of the Congo,2020,1356.0,1000-3000
2702,NGA,Nigeria,2020,2102.0,1000-3000


In [13]:
import pandas as pd

kill_data = data.groupby(['country', 'country_txt', 'iyear'])['nkill'].sum().reset_index()
kill_data.to_csv('what2.csv')
all_countries = kill_data['country'].unique()
all_years = kill_data['iyear'].unique()

all_combinations = pd.MultiIndex.from_product([all_countries, all_years], names=['country', 'iyear']).to_frame(index=False)
complete_data = pd.merge(all_combinations, kill_data, on=['country', 'iyear'], how='left')
complete_data['nkill'] = complete_data['nkill'].fillna(0)



country_names = kill_data[['country', 'country_txt']].drop_duplicates()
complete_data = pd.merge(complete_data, country_names, on='country', how='left')
complete_data = complete_data.sort_values(['country', 'iyear']).reset_index(drop=True)

complete_data.drop('country_txt_x', axis=1)
complete_data = complete_data.rename(columns={'country_txt_y': 'Country'})

log_bin_edges = [-1, 3, 10, 30, 100, 300, 1000, 3000, 10000, 30000]
log_bin_labels = ['-1-3', '3-10', '10-30', '30-100', '100-300', '300-1000', '1000-3000', '3000-10000', '10000-30000']
complete_data['nkill_log'] = pd.cut(complete_data['nkill'], bins=log_bin_edges, labels=log_bin_labels, include_lowest=True)

complete_data = complete_data.sort_values(['iyear', 'nkill']).reset_index()

# complete_data.to_csv('what.csv')

complete_data

,index,country,iyear,country_txt_x,nkill,Country,nkill_log
0,0,AFG,1970,NaN,0.0,Afghanistan,-1-3
1,50,AGO,1970,NaN,0.0,Angola,-1-3
2,100,ALB,1970,NaN,0.0,Albania,-1-3
3,150,AND,1970,NaN,0.0,Andorra,-1-3
4,200,ARE,1970,NaN,0.0,United Arab Emirates,-1-3
...,...,...,...,...,...,...,...
10240,8744,SYR,2020,Syria,973.0,Syria,300-1000
10241,3044,ETH,2020,Ethiopia,1330.0,Ethiopia,1000-3000
10242,1799,COD,2020,Democratic Republic of the Congo,1356.0,Democratic Republic of the Congo,1000-3000
10243,6694,NGA,2020,Nigeria,2102.0,Nigeria,1000-3000


In [6]:
import plotly.express as px
import numpy as np

fig = px.choropleth(complete_data,
                    locations='country',
                    color='nkill_log',
                    hover_data={'Country': True, 'nkill_log': True, 'country': False, 'iyear': False, 'nkill_log': True},
                    range_color=(0, np.max(complete_data['nkill_log'])),
                    animation_frame='iyear',
                    color_continuous_scale='hot',
                    color_discrete_sequence=['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921'],
                    labels={'nkill_log':'Total number of Fatalities'},
                    template='plotly_dark'
)

sliders=[{
    "active": 0,
    "currentvalue": {"prefix": "Year "},
    "steps": fig.layout.sliders[0].steps,
}]


fig.update_layout(
    plot_bgcolor='black',
    margin={"r":20,"t":10,"l":20,"b":0},
    sliders=sliders,
)

fig.update_traces(marker_line_width=0)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 120
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30

# to remove play button
# fig["layout"].pop("updatemenus")

fig.show()



In [7]:
fig = px.choropleth(complete_data,
                    locations='country',
                    color='nkill',
                    hover_data={'Country': True, 'nkill': True, 'country': False, 'iyear': False, 'nkill': True},
                    range_color=(0, np.max(complete_data['nkill'])),
                    animation_frame='iyear',
                    # color_continuous_scale='YlOrRd_r',
                    color_continuous_scale='Reds_r',
                    color_discrete_sequence=['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921'],
                    labels={'nkill':'Total number of Fatalities'},
                    template='plotly_dark'
)

sliders=[{
    "active": 0,
    "currentvalue": {"prefix": "Year "},
    "steps": fig.layout.sliders[0].steps,
}]


fig.update_layout(
    plot_bgcolor='black',
    margin={"r":20,"t":10,"l":20,"b":0},
    sliders=sliders,
)

fig.update_traces(marker_line_width=0)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 120
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 30

# to remove play button
# fig["layout"].pop("updatemenus")

fig.show()

